<h1>Reconocimiento de audio</h1>

<b>Disclaimer: Este notebook utiliza funciones que estan definidos en distintos archivos .py que estan incluidos en la carpeta del proyecto <b>

<h2>Importacion de librerias necesarias</h2>

In [2375]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from tensorflow.keras.regularizers import l2
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC

from sklearn.svm import SVC
from imblearn.over_sampling import SMOTE
import parselmouth
from audioLoading import loadAudios
from scipy.io import wavfile
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt, IPython.display as ipd
import librosa, librosa.display
from convertFiles import convertFiles
from pathlib import Path
from scipy.io import wavfile

Cargar los audios

In [1944]:
audios = loadAudios(f"./AUDIO/All-Audios/Metodo-01_Denoised/*")
audios_features = []

In [1924]:
print(audios)

[[(array([-1.3313080e-05, -9.3354274e-06, -9.7812981e-06, ...,
       -2.0399716e-04, -2.0335178e-04, -8.2252896e-05], dtype=float32), 22050), 'AdriaM-03_1.mp3'], [(array([-1.5522096e-07, -2.0305548e-07, -2.5823735e-07, ...,
       -6.2920004e-03, -6.8021575e-03, -2.8185856e-03], dtype=float32), 22050), 'AdriaM-03_2.mp3'], [(array([-5.2697473e-08, -1.5160907e-07, -2.0447078e-07, ...,
        1.6487477e-04,  1.4839305e-04,  5.5636774e-05], dtype=float32), 22050), 'AdriaM-03_3.mp3'], [(array([-2.4270494e-07, -2.5347794e-07, -1.6956022e-07, ...,
       -3.4186192e-04, -2.0765293e-04, -4.9369781e-05], dtype=float32), 22050), 'AdriaM-03_4.mp3'], [(array([7.8834290e-08, 1.0038820e-07, 9.6849071e-08, ..., 7.1847008e-04,
       6.8352307e-04, 2.6229228e-04], dtype=float32), 22050), 'AdriaM-03_5.mp3'], [(array([ 0.        ,  0.        ,  0.        , ..., -0.00118277,
        0.00132469,  0.00157111], dtype=float32), 22050), 'AleM-03_1.mp3'], [(array([ 0.        ,  0.        ,  0.        , ..., 

Extraccion de caracteristicas

In [2443]:
audios_features = []
labels =[]
for audio_signal,filename in audios:
    features = [
        np.mean(librosa.feature.mfcc(y=audio_signal[0], sr=audio_signal[1], n_mfcc=13)[0,0]),
        librosa.feature.spectral_contrast(y=audio_signal[0],sr=audio_signal[1])[0,0],
        librosa.feature.chroma_stft(y=audio_signal[0],sr=audio_signal[1])[0,0],
        librosa.feature.zero_crossing_rate(y=audio_signal[0])[0,0],
        librosa.feature.rms(y=audio_signal[0])[0,0],
        librosa.feature.spectral_rolloff(y=audio_signal[0])[0,0],
        np.mean(parselmouth.Sound(audio_signal[0]).to_harmonicity().values),
        np.mean(librosa.yin(y=audio_signal[0], fmin=80, fmax=300)),
        librosa.feature.delta(librosa.feature.mfcc(y=audio_signal[0], sr=audio_signal[1], n_mfcc=13), order=1)[0,0],
        librosa.feature.delta(librosa.feature.mfcc(y=audio_signal[0], sr=audio_signal[1], n_mfcc=13), order=2)[0,0]
    ]
    labels.append(str.split(filename,'-')[0])
    audios_features.append(features)

Crear el DataFrame

In [2441]:
df = pd.DataFrame(audios_features,columns=["MFFC","spectral_contrast","chroma_stft","zero_crossing_rate","rms","Spectral Rolloff","Harmonic Sound","Fourier","f0","MFCC Velocity","MFCC Acceleration"])
df["Label"]=labels
df = df[["Label"]+list(df.columns[:-1])]
df.to_csv("audio_features.csv",index=False)

ValueError: 11 columns passed, passed data had 10 columns

Establecer X y Y

In [2437]:
x = df[["MFFC","spectral_contrast","chroma_stft","zero_crossing_rate","rms","Spectral Rolloff","Harmonic Sound","Fourier","f0","MFCC Velocity","MFCC Acceleration"]]
y = df["Label"]

Normalizar datos (Usando Standard Scaler)

In [2299]:
scaler = MinMaxScaler(feature_range=(-2,2))
x_scaled = scaler.fit_transform(x)

In [2405]:
pca = PCA(n_components=6)
x_pca=pca.fit_transform(x_scaled)
df_pca=pd.DataFrame(x_pca)

In [2301]:
x_train,x_test,y_train,y_test = train_test_split(x_scaled,y,test_size = 0.2, random_state=42)

Crear y entrenar el modelo (SVM)

In [2427]:
clf = SVC(kernel='rbf', C=10, gamma='scale')
clf.fit(x_train,y_train)

SVC(C=10)

In [2429]:
accuracy = clf.score(x_test,y_test)
print(accuracy)

0.4583333333333333


In [2431]:
cv_scores = cross_val_score(clf,x,y,cv=4)

In [2421]:
print(cv_scores)

[0.13333333 0.18888889 0.20224719 0.13483146]


Crear y entrenar el modelo (Random Forest)

In [2371]:
model = RandomForestClassifier(n_estimators=100, max_depth=32, random_state=42)
model.fit(x_train,y_train)

RandomForestClassifier(max_depth=32, random_state=42)

In [2373]:
y_pred = model.predict(x_test)
accuracy = accuracy_score(y_test,y_pred)
print(accuracy)

0.625
